# 1. Importing Modules & Processing Data

The first steep is importing the modules needed for processing the data.

In [1]:
!pip install xgboost catboost pyarrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.1 MB/s eta 0:00:00


In [2]:
!python pipeline_v3.py

=== PIPELINE START ===

Walk-forward feature selection (method: random_forest, top_n: 20)...
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)

Feature selection for window 1/4
  Top 10 selected features: ['net_debt_to_equity', 'dsi', 'reps41', 'ev_to_ebit', 'epsp', 'repsfs', 'gross_profit_margin', 'ddtcn', 'reps1', 'epsf']
  Example feature scores (top 5): [np.float64(0.043823828686907824), np.float64(0.04169684577599483), np.float64(0.03948644013462485), np.float64(0.03925614374176234), np.float64(0.03847183150104957)]

Feature selection for window 2/4
  Top 10 selected features: ['repsfs', 'epsp', 'repsf4', 'short_interest', 'repsfl', 'sue', 'price_to_book', 'ddtcn', 'epsf', 'net_debt_to_equity']
  Example feature scores (top 5): [np.float64(0.06444261912823934), np.float64(0.04907874454816956), np.float64(0

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
import tensorflow as tf

Importing the data, check the data by applying the function head, then cleaning the data.

In [ ]:
X_train = pd.read_parquet("X_train.parquet")
X_train.head()

,stock_id,date,piot_norm,dsi,dtcn,ddtcn,short_interest,asset_turnover_ratio,current_liabilities,ev_to_ebit,...,deps,value_mean_eps,value_smart_eps,value_split_adj_mean_eps,value_split_adj_smart_eps,MASK_EARNINGS,gics_sector,gics_group,gics_industry,gics_subindustry
0,1,2014-02-25,NaN,NaN,NaN,NaN,1406440.0,NaN,NaN,NaN,...,NaN,0.240994,NaN,0.49452,NaN,1,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure","Hotels, Resorts & Cruise Lines"
1,1,2014-05-07,0.875,NaN,NaN,NaN,2216110.0,0.236071,2.090000e+09,24.9368,...,-0.00147,0.125585,0.124868,0.25770,0.25623,1,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure","Hotels, Resorts & Cruise Lines"
2,1,2014-07-30,0.875,NaN,NaN,NaN,2183770.0,0.236071,2.090000e+09,24.9368,...,0.00150,0.277427,0.278158,0.56928,0.57078,1,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure","Hotels, Resorts & Cruise Lines"
3,1,2014-10-29,0.875,NaN,NaN,NaN,4279720.0,0.236071,2.090000e+09,24.9368,...,0.00726,0.243772,0.247632,0.50022,0.50814,1,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure","Hotels, Resorts & Cruise Lines"
4,1,2015-02-13,0.875,0.004025,1.134529,-0.362399,1931410.0,0.236071,2.090000e+09,24.9368,...,0.00741,0.261184,0.264795,0.53595,0.54336,1,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure","Hotels, Resorts & Cruise Lines"


In [ ]:
DummyX_test = pd.read_parquet("DummyX_test.parquet")
DummyX_test.head()

,stock_id,date,piot_norm,dsi,dtcn,ddtcn,short_interest,asset_turnover_ratio,current_liabilities,ev_to_ebit,...,deps,value_mean_eps,value_smart_eps,value_split_adj_mean_eps,value_split_adj_smart_eps,MASK_EARNINGS,gics_sector,gics_group,gics_industry,gics_subindustry
33,1,2022-04-29,0.559694,0.029782,3.374708,-8.124451,1.495047e+07,1.895862,2.378440e+09,124.073778,...,-0.119742,1.329654,0.880482,0.961276,0.846399,1.0,Communication Services,Consumer Durables & Apparel,Consumer Finance,Timber REITs
34,1,2022-07-25,0.578449,0.010995,15.544933,-4.274415,1.850066e+07,1.337111,1.169435e+09,157.346782,...,-0.020472,2.559878,10.052196,0.254823,-0.064558,1.0,Consumer Staples,Software & Services,Hotel & Resort REITs,Fertilizers & Agricultural Chemicals
35,1,2022-10-24,0.644343,0.161878,12.139300,3.901776,-3.592270e+05,1.484437,9.942401e+10,NaN,...,-0.007814,1.315210,0.076584,2.063647,-0.745158,1.0,Financials,Utilities,Containers & Packaging,Electronic Equipment & Instruments
36,1,2023-02-07,0.585925,0.030798,1.942343,1.305056,-9.286998e+04,0.058715,4.632626e+09,NaN,...,0.055312,-0.050804,0.398179,0.825654,0.448080,1.0,Energy,Consumer Services,Broadline Retail,Electronic Equipment & Instruments
37,1,2023-04-24,0.680869,0.016672,0.610124,-1.093660,5.246288e+06,0.407839,1.305684e+09,80.983611,...,0.148465,0.229002,0.140618,0.133723,0.538057,1.0,Consumer Staples,Technology Hardware & Equipment,Mortgage Real Estate Investment Trusts (REITs),Leisure Facilities


In [ ]:
y_train = pd.read_parquet("y_train.parquet")
y_train.head()

,stock_id,date,adjusted_close,auxiliary_target_1,auxiliary_target_2,forward_return
0,1,2014-02-25,43.4775,0.000000,-3.517415,-0.002219
1,1,2014-05-07,44.0411,0.402140,-2.724038,0.018992
2,1,2014-07-30,47.9671,0.114653,-3.088564,0.003305
3,1,2014-10-29,47.7339,0.033862,-3.151757,0.009196
4,1,2015-02-13,55.5276,-0.062309,-3.400788,-0.003840


Then we need to clean the datasets

# 2. Training for models